In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from neuralprophet import NeuralProphet
import yfinance as yf
import os
import re

import pandas as pd
import pdfplumber
import requests
import gensim
import nltk

from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from collections import namedtuple
from bs4 import BeautifulSoup as bs
# from gensim.models import CoherenceModel


In [ ]:
df = pd.read_csv('finalp6apr/data.csv')

In [ ]:
df

In [ ]:
def count_words_with_order(words):
    word_count_dict = {}
    for word in words:
        word_count_dict[word] = word_count_dict.get(word, 0) + 1
    return pd.Series(word_count_dict)

word_counts_per_paper = df.dropna().groupby('Paper_ID')['Token_Text'].apply(lambda x: x.str.split().apply(count_words_with_order)).reset_index()


word_counts_per_paper = word_counts_per_paper.melt(id_vars='Paper_ID', var_name='Word', value_name='Count').dropna()
unwanted_words = [r'^\d+( \d+)*$',
                  r'\b\w*[0-9]+\b',
                  r'© UCLES',r'cid',
                  r'\.{10,}',r'^\.{3,}',
                  r'^© UCLES \d+\s*$',
                  r'^\[Total:\s*\d+\]$',
                  r'^\d+\s*(BLANK PAGE|TURN OVER|Additional Pages)',
                  r'^\d+\s*[A-Z]+\s*\d+/\d+/\d+','scale','km','mm',
                  'type','cm','one','impact','level_1','Study','map',
                  'area','shown','feature','extract','Describe','two',
                  'east','north','south','west','Table','grid','Ballvaghan']
word_counts_per_paper['Word'] = word_counts_per_paper['Word'].apply(lambda x: '' if x in unwanted_words else x)


word_counts_per_paper = word_counts_per_paper[word_counts_per_paper['Word'] != '']


print(word_counts_per_paper.head())



for paper_id, paper_data in word_counts_per_paper.groupby('Paper_ID'):
    plt.figure(figsize=(10, 6))
    plt.plot(paper_data['Word'], paper_data['Count'], marker='o', linestyle='-', label=paper_id)
    plt.title('Word Counts for Paper: ' + paper_id)
    plt.xlabel('Word')
    plt.ylabel('Count')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()



In [ ]:
from geotext import GeoText
def count_words_with_order(words):
    word_count_dict = {}
    for word in words:
        word_count_dict[word] = word_count_dict.get(word, 0) + 1
    return pd.Series(word_count_dict)

word_counts_per_paper = df.dropna().groupby('Paper_ID')['Token_Text'].apply(lambda x: x.str.split().apply(count_words_with_order)).reset_index()
word_counts_per_paper = word_counts_per_paper.melt(id_vars='Paper_ID', var_name='Word', value_name='Count').dropna()

unwanted_words = ['scale','km','mm','type','cm','one','impact','level_1','Study','map','area','shown','feature',
                  'extract','Describe','two','east','north','south','west','Table', 'grid','Ballyvaghan']

word_counts_per_paper['Word'] = word_counts_per_paper['Word'].apply(lambda x: '' if any(char.isdigit() for char in x) or x in unwanted_words else x)
word_counts_per_paper = word_counts_per_paper[word_counts_per_paper['Word'] != '']

for paper_id, paper_data in word_counts_per_paper.groupby('Paper_ID'):
    plt.figure(figsize=(10, 6))
    plt.plot(paper_data['Word'], paper_data['Count'], marker='o', linestyle='-', label=paper_id)
    plt.title('Word Counts for Paper: ' + paper_id)
    plt.xlabel('Word')
    plt.ylabel('Count')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
import nltk
from nltk.corpus import wordnet as wn

# Download the wordnet corpus if not already downloaded
nltk.download('wordnet')

# Function to check if a word is a city or country name
def is_city_or_country(word):
    synsets = wn.synsets(word)
    for synset in synsets:
        if synset.lexname() == 'toponym':
            return True
    return False

# Extract city and country names from text
def extract_places(text):
    words = nltk.word_tokenize(text)
    places = [word for word in words if is_city_or_country(word)]
    return places

# Apply the function to extract places for each row in the DataFrame
places = df['Token_Text'].dropna().apply(extract_places)

# Flatten the list of lists into a single list of places
all_places = [place for sublist in places for place in sublist]

# Convert the list to a set to remove duplicates
unique_places = set(all_places)

# Convert the set back to a list
unwanted_words = list(unique_places)

# Filter out unwanted words
word_counts_per_paper['Word'] = word_counts_per_paper['Word'].apply(lambda x: '' if any(char.isdigit() for char in x) or x in unwanted_words else x)

# Filter out rows with empty words
word_counts_per_paper = word_counts_per_paper[word_counts_per_paper['Word'] != '']

print(word_counts_per_paper.head())

for paper_id, paper_data in word_counts_per_paper.groupby('Paper_ID'):
    plt.figure(figsize=(10, 6))
    plt.plot(paper_data['Word'], paper_data['Count'], marker='o', linestyle='-', label=paper_id)
    plt.title('Word Counts for Paper: ' + paper_id)
    plt.xlabel('Word')
    plt.ylabel('Count')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
# df = pd.DataFrame(data)

# Function to count words within each group
def count_words(group):
    word_counts = group.str.split().explode().value_counts()
    return word_counts

# Apply the count_words function within each group defined by Paper_ID
word_counts_per_paper = df.groupby('Paper_ID')['Token_Text'].apply(count_words).reset_index()

# Rename columns
word_counts_per_paper.columns = ['Paper_ID', 'Word', 'Count']

# Display the resulting DataFrame
print(word_counts_per_paper)

In [ ]:
for paper_id, paper_data in word_counts_per_paper.groupby('Paper_ID'):
    plt.figure(figsize=(10, 6))
    plt.plot(paper_data['Word'], paper_data['Count'], marker='o', linestyle='-', label=paper_id)
    plt.title('Word Counts for Paper: ' + paper_id)
    plt.xlabel('Word')
    plt.ylabel('Count')
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()